In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

plt.style.use('seaborn')
sns.set_palette('cubehelix')
plt.rcParams['figure.figsize'] = [5, 3.5]
plt.rcParams['figure.dpi'] = 150
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
N_PORTFOLIOS = 10 ** 5 
N_DAYS = 252 
RISKY_ASSETS = ['AAPL', 'IBM', 'MSFT']
RISKY_ASSETS.sort()
START_DATE = '2018-01-01'
END_DATE = '2018-12-31'

n_assets = len(RISKY_ASSETS)

In [ ]:
prices_df = yf.download(RISKY_ASSETS, start=START_DATE, 
                        end=END_DATE, adjusted=True)
print(f'Downloaded {prices_df.shape[0]} rows of data.')

In [ ]:
returns_df = prices_df['Adj Close'].pct_change().dropna()

avg_returns = returns_df.mean() * N_DAYS
cov_mat = returns_df.cov() * N_DAYS

In [ ]:
np.random.seed(42)
weights = np.random.random(size=(N_PORTFOLIOS, n_assets))

weights /= np.sum(weights, axis=1)[:,np.newaxis]

In [ ]:
portf_rtns = np.dot(weights,avg_returns)

portf_vol = []
for i in range(0, len(weights)): 
    portf_vol.append(np.sqrt(np.dot(weights[i].T, 
                                    np.dot(cov_mat, weights[i]))))
  

portf_vol = np.array(portf_vol) 
portf_sharpe_ratio = portf_rtns / portf_vol 

In [ ]:
portf_results_df = pd.DataFrame({'returns': portf_rtns, 
                                 'volatility': portf_vol, 
                                 'sharpe_ratio': portf_sharpe_ratio})

In [ ]:
max_sharpe_ind = np.argmax(portf_results_df.sharpe_ratio)

max_sharpe_portf = portf_results_df.loc[max_sharpe_ind]

min_vol_ind = np.argmin(portf_results_df.volatility)

min_vol_portf = portf_results_df.loc[min_vol_ind]

In [ ]:
print('Maximum Sharpe Ratio portfolio ----')
print('Performance')
for index, value in max_sharpe_portf.items():
    print(f'{index}: {100 * value:.2f}% ', end="", flush=True)
print('\nWeights')
for x, y in zip(RISKY_ASSETS, weights[np.argmax(portf_results_df.sharpe_ratio)]):
    print(f'{x}: {100*y:.2f}% ', end="", flush=True)

In [ ]:
import scipy
import scipy.linalg

In [ ]:
min_var_portf_w = weights[np.argmin(portf_results_df.volatility)]

In [ ]:
T = 1
N = len(returns_df)
N_SIM = 10**3

In [ ]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, 
                 random_seed=42):
      np.random.seed(random_seed)

    dt = T/N 
    dW = np.random.normal(scale = np.sqrt(dt), size=(n_sims, N))
    W = np.cumsum(dW, axis =1)

    time_step = np.linspace(dt, T, N)
    time_steps = np.broadcast_to(time_step, (n_sims, N))

    S_t = s_0 * np.exp((mu - 0.5 * sigma ** 2) * time_steps
                       + sigma * W)
    S_t = np.insert(S_t, 0, s_0, axis=1)

    return S_t

In [ ]:
shares = max_sharpe_portf_w
print(shares)

In [ ]:
mu = np.matmul(avg_returns,shares)
sigma = np.matmul(np.matmul(shares,cov_mat.values),shares.T)
print(mu)
print(sigma)

In [ ]:
gbm_simulations = simulate_gbm(1, mu, sigma, N_SIM, T, N)

In [ ]:
plot_title = (f'{RISKY_ASSETS} Simulation ', 'Maximum Sharpe Ratio')


gbm_simulations_df = pd.DataFrame(np.transpose(gbm_simulations))

ax = gbm_simulations_df.plot(alpha=0.2, legend=False)
line_1 = ax.plot(gbm_simulations_df.mean(axis=1), 
                  color='red')

ax.set_title(plot_title, fontsize=16)
ax.legend((line_1), ('m'))


plt.show()

In [ ]:
shares = min_var_portf_w

In [ ]:
mu = np.matmul(avg_returns,shares)
sigma = np.matmul(np.matmul(shares,cov_mat.values),shares.T)
print(mu)
print(sigma)

In [ ]:
plot_title = (f'{RISKY_ASSETS} Simulation ', 'Minimum Volatility')


gbm_simulations_df = pd.DataFrame(np.transpose(gbm_simulations))


ax = gbm_simulations_df.plot(alpha=0.2, legend=False)
line_1 = ax.plot(gbm_simulations_df.mean(axis=1), 
                  color='red')

ax.set_title(plot_title, fontsize=16)
ax.legend((line_1), ('m'))


plt.show()